In [1]:
import os

import numpy as np
from sklearn.linear_model import LogisticRegression

from utils import load_hidden_representations_from_hdf5, read_templates_from_file

----

In [4]:
# params
log_dir = "/logfiles"
model = "bigscience-T0_3B" # bigscience-T0_B or bigscience-T0
module = "encoder" # encoder
# task = "rte"
task = "cb"
# task = "wic"

In [3]:
assert module == "encoder" # TODO(mm): support decoder as well

## Prompts

In [5]:
df = read_templates_from_file(f"/t0-analysis/prompts/{task}.csv")
display(df)

,name,template,category,includes_labels,shuffle
0,null_pattern,{premise} {hypothesis},neutral,False,False
1,null_pattern_reversed,{hypothesis} {premise},neutral,False,False
2,gpt_3_true_false_neither,"{premise} Question: {hypothesis} True, False, ...",instructive,True,False
3,gpt_3_yes_no_maybe,"{premise} Question: {hypothesis} Yes, No, or M...",instructive,True,False
4,mnli_crowdsource,{premise} Using only the above description and...,instructive,True,False
5,always_sometimes_never,"Suppose it's true that {premise} Then, is ""{hy...",instructive,True,False
6,based_on_previous_passage,"{premise} Based on the previous passage, is it...",instructive,True,False
7,infer,"Suppose {premise} Can we infer that ""{hypothes...",instructive,True,False
8,claim,"{premise} Based on that information, is the cl...",instructive,True,False
9,consider,"{premise} Keeping in mind the above text, cons...",instructive,True,False


## Train linear classifier

In [6]:
def unison_shuffled_copies(a, b):
    # from: https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [7]:
# RTE patterns
# use_pattern = [
#     "null_pattern",
#     "null_pattern_reversed",
#     "gpt_3_yes_no",
#     "gpt_3_yes_no_shuffled",
#     "gpt_3_true_false",
#     "gpt_3_true_false_shuffled",
#     "start_with_the",
#     "mnli_crowdsource",
#     "based_on_previous_passage",
#     "infer",
#     "follow",
#     "imply",
#     "guaranteed",
#     "justified", 
#     "must_be_true",
#     "should_assume"
# ]

# CB patterns
use_pattern = [
    # "null_pattern",
    # "null_pattern_reversed",
    "gpt_3_true_false_neither",
    "gpt_3_yes_no_maybe",
    # "mnli_crowdsource",
    # "always_sometimes_never",
    # "based_on_previous_passage",
    # "infer",
    # "claim",
    # "consider",
    # "follow",
    # "imply",
    # "guaranteed",
    # "guaranteed_possible",
    # "justified",
    # "must_be_true",
    # "should_assume",
    # "take_the_following",
]

# WIC patterns
# use_pattern = [
#     "gpt_3",
#     "gpt_3_yes_no",
#     "affirmation",
#     "grammar_homework",
#     "polysemous",
#     "question_context",
#     "question_meaning",
#     "question_meaning_yes_no",
#     "same_sense",
#     "similar_sense",
#     "similar_sense_yes_no",
# ]

In [11]:
# for layer in range(0, 10):
for layer in range(0, 25):
# for layer in range(24, 25):
    
    print("layer:", layer)
    file_names, prompt_names = [], []

    for _, row in df.iterrows():
        if row['name'] in use_pattern:
            file_names.append(f"{task}/{model}/{module}/{row['name']}/hidden_represenations_layer{layer}_avg.hdf5",)
            prompt_names.append(row['name'])

    # load hidden representations from hdf5 file
    representations = None
    classes = []
    n_sequences = 0

    for idx, file_name in enumerate(file_names):
        hidden_representations = load_hidden_representations_from_hdf5(os.path.join(log_dir, file_name))
        # print(hidden_representations.shape)
        n_sequences = hidden_representations.shape[0]

        if representations is None:
            representations = hidden_representations
        else:
            representations = np.concatenate((representations, hidden_representations), axis=0)

        classes += n_sequences * [idx] # assign representations to classes
    
    classes = np.asarray(classes)

    # shuffle representations and classes
    X, y = unison_shuffled_copies(representations, classes)
    print(X.shape, y.shape)

    # train linear classifier
    # multi_class='multinomial' uses a CE loss
    print('classifying between:', prompt_names)
    clf = LogisticRegression(random_state=0, max_iter=2000, multi_class='multinomial').fit(X, y)
    
    print(f'layer={layer}; accuracy on training data: ', clf.score(X, y))
    print('\n')



layer: 0


Reading embeddings: 100%|██████████| 56/56 [00:00<00:00, 4952.68it/s]


(112, 2048) (112,)
classifying between: ['gpt_3_true_false_neither', 'gpt_3_yes_no_maybe']
layer=0; accuracy on training data:  1.0


layer: 1


Reading embeddings: 100%|██████████| 56/56 [00:00<00:00, 4269.63it/s]

(112, 2048) (112,)
classifying between: ['gpt_3_true_false_neither', 'gpt_3_yes_no_maybe']


...

Reading embeddings: 100%|██████████| 56/56 [00:00<00:00, 3542.81it/s]

(112, 2048) (112,)
classifying between: ['gpt_3_true_false_neither', 'gpt_3_yes_no_maybe']


layer=23; accuracy on training data:  1.0


layer: 24


Reading embeddings: 100%|██████████| 56/56 [00:00<00:00, 3413.87it/s]

(112, 2048) (112,)
classifying between: ['gpt_3_true_false_neither', 'gpt_3_yes_no_maybe']
layer=24; accuracy on training data:  0.9821428571428571


